In [2]:
import pandas as pd

In [1]:
import pandas as pd

# --- 1. Cargar archivos ---
df_median = pd.read_csv("./grupo/lgbm_predictions_median.csv").rename(columns={"product_id": "PRODUCT_ID"})
df_reg = pd.read_csv("reg_lineal5.csv").rename(columns={"product_id": "PRODUCT_ID", "tn": "reg_lineal"})
df_lightgbm10 = pd.read_csv("./grupo/lightgbm10.csv").rename(columns={"product_id": "PRODUCT_ID"})
df_predicciones_exp_07_lgb_v8 = pd.read_csv("./grupo/predicciones_exp_07_lgb_v8.csv").rename(columns={"product_id": "PRODUCT_ID"})
df_predicciones_febrero2020_porproducto3 = pd.read_csv("./grupo/predicciones_febrero2020_porproducto3.csv").rename(columns={"product_id": "PRODUCT_ID"})



In [3]:
import pandas as pd

# --- Cargar archivos con nombres únicos ---
df_median = pd.read_csv("./grupo/lgbm_predictions_median.csv").rename(columns={"tn": "tn_median"})
df_lgb10 = pd.read_csv("./grupo/lightgbm10.csv").rename(columns={"tn": "tn_lgb10"})
df_exp07 = pd.read_csv("./grupo/predicciones_exp_07_lgb_v8.csv").rename(columns={"tn": "tn_exp07"})
df_ens3 = pd.read_csv("./grupo/predicciones_exp_07_lgb_v8_ensemble_3models.csv").rename(columns={"tn": "tn_ens3"})
df_febrero = pd.read_csv("./grupo/predicciones_febrero2020_porproducto3.csv").rename(columns={"tn_predicho": "tn_febrero"})


In [ ]:

# --- Unir todos por PRODUCT_ID ---
df_all = df_median.merge(df_lgb10, on="product_id", how="inner") \
                  .merge(df_exp07, on="product_id", how="inner") \
                  .merge(df_ens3, on="product_id", how="inner") \
                  .merge(df_febrero, on="product_id", how="inner")

# --- Calcular estadísticas por fila ---
cols_pred = ["tn_median", "tn_lgb10", "tn_exp07", "tn_ens3", "tn_febrero"]
df_all["tn_mean"] = df_all[cols_pred].mean(axis=1)
df_all["tn_median_all"] = df_all[cols_pred].median(axis=1)
df_all["tn_std"] = df_all[cols_pred].std(axis=1)
df_all["tn_range"] = df_all[cols_pred].max(axis=1) - df_all[cols_pred].min(axis=1)

# --- Guardar resultado ---
df_all.to_csv("df_all.csv", index=False)
print("✅ Archivo guardado como df_all.csv")

import pandas as pd

# --- 1. Cargar el archivo generado ---
# df_all = pd.read_csv("df_all.csv")

# --- 2. Asegurarse de que estas columnas existen ---
# Si ya las calculaste en otro script, podés omitir esto
if "tn_range" not in df_all.columns or "tn_mean" not in df_all.columns:
    pred_cols = ["tn_median", "tn_lgb10", "tn_exp07", "tn_ens3", "tn_febrero"]
    df_all["tn_mean"] = df_all[pred_cols].mean(axis=1)
    df_all["tn_range"] = df_all[pred_cols].max(axis=1) - df_all[pred_cols].min(axis=1)

# --- 3. Calcular dispersión relativa ---
df_all["range_pct"] = df_all["tn_range"] / (df_all["tn_mean"] + 1e-6)

# --- 4. Filtrar productos con alta dispersión (> 20%) ---
df_dispersos = df_all[df_all["range_pct"] > 0.40].copy()
df_dispersos = df_dispersos.sort_values("range_pct", ascending=False)

# --- 5. Guardar resultados ---
df_dispersos[["product_id", "tn_range", "tn_mean", "range_pct"]].to_csv("productos_con_alta_dispersión.csv", index=False)

# --- 6. Mostrar resumen ---
print(f"✅ Se encontraron {len(df_dispersos)} productos con dispersión > 40%")
print("Archivo guardado: productos_con_alta_dispersión.csv")


✅ Archivo guardado como df_all.csv


In [12]:
import pandas as pd

# --- Si ya tenés df_all en memoria, no hace falta volver a cargar ---
# df_all = pd.read_csv("df_all.csv")

# --- 1. Verificamos columnas necesarias ---
pred_cols = ["tn_median", "tn_lgb10", "tn_exp07", "tn_ens3", "tn_febrero"]
if "tn_range" not in df_all.columns:
    df_all["tn_range"] = df_all[pred_cols].max(axis=1) - df_all[pred_cols].min(axis=1)
if "tn_mean" not in df_all.columns:
    df_all["tn_mean"] = df_all[pred_cols].mean(axis=1)

# --- 2. Filtrar por dispersión absoluta > 30 toneladas ---
df_dispersos = df_all[df_all["tn_range"] > 40].copy()
df_dispersos = df_dispersos.sort_values("tn_range", ascending=False)

# --- 3. Guardar resultados ---
df_dispersos[["product_id", "tn_range", "tn_mean"]].to_csv("productos_con_dispersión_mayor_50ton.csv", index=False)

# --- 4. Mostrar resumen ---
print(f"✅ Se encontraron {len(df_dispersos)} productos con dispersión > 50 toneladas")
print("Archivo guardado: productos_con_dispersión_mayor_50ton.csv")


✅ Se encontraron 71 productos con dispersión > 50 toneladas
Archivo guardado: productos_con_dispersión_mayor_50ton.csv
